In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pvlib
from pvlib.irradiance import aoi
import numpy as np

In [2]:
df_France= pd.read_excel('VK France/VKR France Report_All data fall 2023.xlsx')

In [3]:
#removes the two first rows
df_France = df_France.iloc[2:].reset_index(drop=True)

In [4]:
df_France = df_France.drop(columns=['Black out'])

In [5]:
df_France = df_France.rename(columns={'Unnamed: 1': 'Time', 'Unnamed: 2': 'In Temp', 'Unnamed: 3': 'Gip Top Center','Unnamed: 4' : 'Gip Bottom Center','Unnamed: 5':'Cladding uPV East'	})

In [ ]:
df_France= df_France.drop(columns=['Cladding uPV East'])

In [ ]:
df_France.info()

In [7]:
df_France['Time'] = pd.to_datetime(df_France['Time'], format='%d-%m-%Y %H:%M:%S').dt.strftime("%Y-%m-%d %H:%M")

In [8]:
df_France['Time'] = pd.to_datetime(df_France['Time'])

In [9]:
df_France = df_France.sort_values(by='Time', ascending=True)

In [10]:
df_cleaned= df_France.dropna()

In [11]:
df_cleaned=df_cleaned.set_index('Time')

In [12]:
df_cleaned = df_cleaned.apply(pd.to_numeric, errors='coerce')

In [13]:
df_cleaned.head(20)

,In Temp,Gip Top Center,Gip Bottom Center,Cladding uPV East
Time,,,,
2022-07-07 15:36:00,22.5,22.3,23.1,23.3
2022-07-07 15:41:00,23.3,22.6,23.4,23.4
2022-07-07 15:46:00,23.4,22.7,23.4,23.4
2022-07-07 15:51:00,23.4,22.8,23.4,23.4
2022-07-07 15:56:00,23.4,22.9,23.4,23.4
2022-07-07 16:01:00,23.3,22.9,24.1,23.5
2022-07-07 16:06:00,23.3,22.9,24.2,23.4
2022-07-07 16:11:00,23.2,23.0,23.7,23.5
2022-07-07 16:16:00,23.2,23.0,23.5,23.5


In [ ]:
for column in df_cleaned.columns:
    plt.figure(figsize=(50, 10))
    plt.plot(df_cleaned.index, df_cleaned[column], marker='o', linestyle='-')
    plt.xlabel("Tid")
    plt.ylabel(column)
    plt.title(f"Tidsserie af {column} ")
    plt.grid()
    plt.show()

In [ ]:
df = pd.read_csv('VK France/VKR France Station_Rue_1-7-22_00-00_1_Year_1744728751_v2.csv', encoding='ISO-8859-1',delimiter=',',skiprows=5)

In [ ]:
df.info()

In [17]:
df=df[['Date & Time', 'Temp - °C','Hum - %','Dew Point - °C' ,'Avg Wind Speed - m/s','Solar Rad - W/m^2']]

In [18]:
df = df.rename(columns={'Date & Time':'Time' ,'Temp - °C': 'Temp Out','Hum - %':'Out Hum','Dew Point - °C':'Dew Pt.' ,'Avg Wind Speed - m/s': 'Wind Speed','Solar Rad - W/m^2': 'Solar Rad.'})

In [ ]:
df.head(20)

In [20]:

df['Time'] = pd.to_datetime(df['Time'], format="%d/%m/%y %H:%M").dt.strftime("%Y-%m-%d %H:%M")


In [21]:
df['Time'] = pd.to_datetime(df['Time'])


In [22]:
df=df.set_index('Time')

In [ ]:
df.head()

In [24]:
df = df.apply(lambda x: x.replace('--', np.nan))

In [25]:
df['Temp Out']=df['Temp Out'].str.replace(',', '.')
df['Out Hum']=df['Out Hum'].str.replace(',', '.')
df['Dew Pt.']=df['Dew Pt.'].str.replace(',', '.')
df['Wind Speed']=df['Wind Speed'].str.replace(',', '.')

In [26]:
df = df.apply(pd.to_numeric, errors='coerce')

In [27]:
df_resampled=df.asfreq('30Min')

In [ ]:
df_resampled

In [29]:
merged_df = pd.merge_asof(df_resampled.sort_index(), 
                          df_cleaned.sort_index(), 
                          on='Time', 
                          direction='nearest',
                          tolerance=pd.Timedelta('1min')) 


In [30]:
France=merged_df.dropna()

In [31]:
France.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3357 entries, 320 to 3676
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Time               3357 non-null   datetime64[ns]
 1   Temp Out           3357 non-null   float64       
 2   Out Hum            3357 non-null   float64       
 3   Dew Pt.            3357 non-null   float64       
 4   Wind Speed         3357 non-null   float64       
 5   Solar Rad.         3357 non-null   float64       
 6   In Temp            3357 non-null   float64       
 7   Gip Top Center     3357 non-null   float64       
 8   Gip Bottom Center  3357 non-null   float64       
 9   Cladding uPV East  3357 non-null   float64       
dtypes: datetime64[ns](1), float64(9)
memory usage: 288.5 KB


In [32]:
France=France.set_index('Time')

In [ ]:
for column in France.columns:
    plt.figure(figsize=(50, 10))
    plt.plot(France.index, France[column], marker='o', linestyle='-')
    plt.xlabel("Tid")
    plt.ylabel(column)
    plt.title(f"Tidsserie af {column} ")
    plt.grid()
    plt.show()

In [34]:
France['Longitude']= 1.58768
France['Latitude']=50.06788

In [ ]:
France.head()

In [36]:
France['Month'] =France.index.month
France["day_of_year"] = France.index.dayofyear

In [37]:
France["Month_sin"] = np.sin(2 * np.pi * France["Month"] / 12)
France["Month_cos"] = np.cos(2 * np.pi * France["Month"] / 12)

France["day_of_year_sin"] = np.sin(2 * np.pi * France["day_of_year"] / 365)
France["day_of_year_cos"] = np.cos(2 * np.pi * France["day_of_year"] / 365)

In [38]:
France["minute_of_day"] =France.index.hour * 60 + France.index.minute

#  1440 min = 24 hours
France["time_sin"] = np.sin(2 * np.pi * France["minute_of_day"] / 1440)
France["time_cos"] = np.cos(2 * np.pi * France["minute_of_day"] / 1440)

In [39]:
France=France.reset_index('Time')

In [40]:

# calculate the solar position
def get_solar_position(row):
    solpos = pvlib.solarposition.get_solarposition(row['Time'], row['Latitude'], row['Longitude'])
    return pd.Series([solpos['azimuth'].values[0], solpos['apparent_elevation'].values[0]])

In [41]:
France[['solar_azimuth', 'solar_elevation']] = France.apply(get_solar_position, axis=1)
France['solar_azimuth_sin'] = np.sin(np.deg2rad(France['solar_azimuth']))
France['solar_azimuth_cos'] = np.cos(np.deg2rad(France['solar_azimuth']))



In [43]:
France['window_tilt'] = 23
France['window_azimuth'] = 185

In [45]:
France['solar_zenith']=  90 - France['solar_elevation']

In [46]:

France['aoi'] = aoi(
    surface_tilt=France['window_tilt'],
    surface_azimuth=France['window_azimuth'],
    solar_zenith= France['solar_zenith'],
    solar_azimuth=France['solar_azimuth']
)

In [47]:
France['cos_aoi'] = np.cos(np.radians(France['aoi']))

In [48]:
France=France.drop(columns=['aoi','solar_zenith'])

In [50]:
France['Location']='France'

In [52]:
targets=[ 'Gip Top Center', 'Gip Bottom Center', 'Cladding uPV East']

dfs = []

for target in targets:
    serie=France.copy()      
    serie= France.drop(columns=[t for t in targets if t != target])
    serie = serie.rename(columns={target: "sensor_temp"})
    serie["sensor_ID"] = target
    serie["timeserie_ID"] =f"France, {target}"
    dfs.append(serie)


df_France = pd.concat(dfs, ignore_index=True)

In [ ]:
df_France.info()

In [54]:
df_France['window_area'] = 118*78

In [55]:
df_France=df_France.set_index('Time')

In [56]:
resampled_groups = []
for ts_id, group in df_France.groupby('timeserie_ID'):

    group_asfreq = group.asfreq("30min")

    group_asfreq['timeserie_ID'] = ts_id

    resampled_groups.append(group_asfreq)

df_resampled = pd.concat(resampled_groups)
df_filtered_timeserie=df_resampled.copy()

In [ ]:
df_filtered_timeserie=df_filtered_timeserie.reset_index('Time')

In [ ]:

df_filtered = df_filtered_timeserie.groupby('sensor_ID').apply(lambda x: x[x['Time'] >= pd.to_datetime('2022-08-01')]).reset_index(drop=True)


In [63]:
df_filtered.to_csv('france.csv', index=False)